In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np


In [15]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=.05))

def model(x,w_h,w_h2,w_o,p_keep_input,p_keep_hidden):
   
    X = tf.nn.dropout(x,p_keep_input)
    h =tf.nn.relu( tf.matmul(X,w_h))
    
    h = tf.nn.dropout(h,p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h,w_h2))
    
    h2 = tf.nn.dropout(h2,p_keep_hidden)
    out = tf.matmul(h2,w_o)
    
    return out
    

In [16]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
X = tf.placeholder("float",[None,784])
Y = tf.placeholder("float",[None,10])

w_h = init_weights([784,625])
w_h2 = init_weights([625,625])
w_o = init_weights([625,10])

p_hidden = tf.placeholder("float")
p_input = tf.placeholder("float")

py_x = model(X,w_h,w_h2,w_o,p_input,p_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x,labels=Y))
train_opt = tf.train.RMSPropOptimizer(.001,.9).minimize(cost)

predict_op = tf.argmax(py_x,1)

In [22]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(10):
        for start,end in zip(range(0,len(trX),128),range(128,len(trX)+1,128)):
            sess.run(train_opt,feed_dict={X:trX[start:end],Y:trY[start:end],p_input:.8,
                                         p_hidden:.5})
        print(i,np.mean(np.argmax(teY,axis=1)==
                       sess.run(predict_op,feed_dict={X:teX,p_input:1.0,p_hidden:1.0})))

0 0.9429
1 0.9642
2 0.9715
3 0.9734
4 0.9753
5 0.9749
6 0.9752
7 0.9767
8 0.9799
9 0.9786
